In [161]:
from myUtils import functions
import copy
import numpy as np
import re
import os.path as osp
import os
import numpy as np
from pprint import pprint
from skimage.transform import rescale, resize
from skimage import data, color, io
import skimage
import PIL 
import scipy
import json

from keras.applications.xception import preprocess_input
from keras.applications.xception import Xception
from keras.models import Model
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
from keras.applications.imagenet_utils import decode_predictions

Using TensorFlow backend.


In [162]:
#hyper parameters
WORD_EMBEDDING_DIM = 300
MAX_SENTANCE_DIM = 92 # TODO: update considering all the captions in the dataset
MAX_PARAGRAPH_DIM = 5
EMBEDDING_FILE_NAME = 'glove.6B.300d.txt'

In [163]:
class text_encoder:
    
    def load_embedings(self):
        embedding_matrix, vocabulary, word_to_index = functions.get_embedding_matrix(EMBEDDING_FILE_NAME, WORD_EMBEDDING_DIM)      
        self.embedding_matrix = embedding_matrix
        self.vocabulary = vocabulary
        self.word_to_index = word_to_index
        
    def get_sentence_embedding(self,sentence):
        words = sentence.split()    
        sentance_embedding =  np.zeros(shape=(MAX_SENTANCE_DIM,WORD_EMBEDDING_DIM))
        mask =  np.zeros(MAX_SENTANCE_DIM)
        i=0;
        for w in words:
            mask[i] = 1
            index = self.word_to_index.get(w,-1)
            if(index != -1):
                sentance_embedding[i] = self.embedding_matrix[index]
            else:
                sentance_embedding[i] = np.zeros(WORD_EMBEDDING_DIM)
            i+=1       
        return  sentance_embedding, mask
    
    def preprocess_paragraph(self, paragraph):
        pat = re.compile(r"([()!',;:?])")
        processed_paragraph = pat.sub(" \\1", paragraph)
        return processed_paragraph
       
    def clean_text(text):
    '''Clean text by removing unnecessary characters and altering the format of words.'''
        text = text.lower()
        text = re.sub(r"i'm", "i am", text)
        text = re.sub(r"he's", "he is", text)
        text = re.sub(r"she's", "she is", text)
        text = re.sub(r"it's", "it is", text)
        text = re.sub(r"that's", "that is", text)
        text = re.sub(r"what's", "that is", text)
        text = re.sub(r"where's", "where is", text)
        text = re.sub(r"how's", "how is", text)
        text = re.sub(r"\'ll", " will", text)
        text = re.sub(r"\'ve", " have", text)
        text = re.sub(r"\'re", " are", text)
        text = re.sub(r"\'d", " would", text)
        text = re.sub(r"\'re", " are", text)
        text = re.sub(r"won't", "will not", text)
        text = re.sub(r"can't", "cannot", text)
        text = re.sub(r"n't", " not", text)
        text = re.sub(r"n'", "ng", text)
        text = re.sub(r"'bout", "about", text)
        text = re.sub(r"'til", "until", text)
        text = re.sub(r"[-()\"#/@;:<>{}`+=~|.!?,]", "", text)
    
        return text
    
    def get_paragraph_embedding(self, paragraph):
        sentences = paragraph.split(".")
        paragraph_embedding = np.zeros(shape=(MAX_PARAGRAPH_DIM,MAX_SENTANCE_DIM,WORD_EMBEDDING_DIM))
        paragraph_mask = np.zeros(shape=(MAX_PARAGRAPH_DIM,MAX_SENTANCE_DIM))
        i = 0;
        for s in sentences:
            processed_sentence = self.preprocess_paragraph(s.strip())
            sentance_embedding, mask = self.get_sentence_embedding(processed_sentence)
            paragraph_embedding[i] =  sentance_embedding
            paragraph_mask[i] = mask
            i+=1
        return paragraph_embedding, paragraph_mask
            
    

In [142]:
class decoder():
    def get_decoder_model():
        return "something"

In [148]:
# en = encoder()
# en.load_embedings()
# y,  mask = en.get_paragraph_embedding("1 2 3 something's something!!. something else. other one this one")
# # print(y)
# print(mask)
# # x, mask = en.get_sentence_embedding("something something blabblublu")
# # print(len(x))
# # print(x[2])
# # print(mask)
# print(y)

In [171]:
class image_encoder:
    def load_image(self,image_path,target_size):
        img = skimage.io.imread(image_path)
        image_resized = skimage.transform.resize(img, target_size, anti_aliasing=True)
        return image_resized
    
    def extract_features(self,directory):
        model = Xception()
        model.layers.pop() 
        model = Model(inputs=model.inputs, outputs=model.layers[-1].output)
#         print(model.summary())
        features = dict()
        i=0;
        for name in os.listdir(directory):
#             try:
                filename = directory + '/' + name
                image = self.load_image(filename, target_size=(299, 299))
                if image.shape == (299, 299, 3):
#                 print(image.shape)
                    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
                    image = preprocess_input(image)
                    feature = model.predict(image, verbose=0)
                    image_id = name.split('.')[0]
                    x = feature.ravel().tolist() 
                    features[name] = x
                    print(i)
                i +=1
#             except:
#                 continue 
        return features

    def generate_image_vector(self,input_file, output_file):
        x = self.extract_features(input_file)#"/Users/vinutahegde/Documents/USC/SEM3/DL/project/ws/images/train/images/train")  
        j = json.dumps(x)
        f = open(output_file, 'w')
        print(j, file=f)
        f.close()

In [173]:
# ie = image_encoder()
# ie.generate_image_vector('/Users/vinutahegde/Documents/USC/SEM3/DL/project/ws/images/train/images/3', "test.json")